In [ ]:
# %load_ext autoreload
# %autoreload 2
import sys
sys.path.insert(0,'/home/michele/lavoro/code/librascal/build/')
#sys.path.insert(0,'/home/nigam/git/librascal_cs/librascal/build/')

In [ ]:
from ase.io import read
import ase
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt

from rascal.representations import SphericalExpansion, SphericalInvariants
from rascal.utils import (get_radial_basis_covariance, get_radial_basis_pca, 
                          get_radial_basis_projections, get_optimal_radial_basis_hypers )
from rascal.utils import radial_basis
from rascal.utils import (WignerDReal, ClebschGordanReal, 
                          spherical_expansion_reshape, spherical_expansion_conjugate,
                    lm_slice, real2complex_matrix, compute_lambda_soap)

This notebook provides examples of the calculation of equivariant density correlation features, using the iterative expression introduced in [Nigam et al., JCP (2020)](http://doi.org/10.1063/5.0021116). It discusses the practicalities of its implementation with real-valued density coefficients, and shows examples of low-body-order invariants and covariants that NICE features generalize.

# Computes spherical expansion coefficients

In [ ]:
import urllib.request
# a collection of distorted ethanol molecules from the ANI-1 dataset 
# (see https://github.com/isayev/ANI1_dataset) with energies and forces computed using DFTB+ 
# (see https://www.dftbplus.org/)
url = 'https://raw.githubusercontent.com/cosmo-epfl/librascal-example-data/833b4336a7daf471e16993158322b3ea807b9d3f/inputs/molecule_conformers_dftb.xyz'
# Download the file from `url`, save it in a temporary directory and get the
# path to it (e.g. '/tmp/tmpb48zma.txt') in the `structures_fn` variable:
structures_fn, headers = urllib.request.urlretrieve(url)
structures_fn

In [ ]:
# Total number of structure to load
N = 100

# load the structures
frames = read(structures_fn,':{}'.format(N))

In [ ]:
spherical_expansion_hypers = {
    "interaction_cutoff": 3,
    "max_radial": 6,
    "max_angular": 4,
    "gaussian_sigma_constant": 0.3,
    "gaussian_sigma_type": "Constant",
    "cutoff_smooth_width": 0.5,
    "radial_basis": "GTO",
}

spex = SphericalExpansion(**spherical_expansion_hypers)

In [ ]:
selframe = frames[8];         # frame and l value used for the test
feat_scaling = 1e6            # just a scaling to make coefficients O(1)
feats = spex.transform(selframe).get_features(spex)
ref_feats = feat_scaling*spherical_expansion_reshape(feats, **spherical_expansion_hypers)

# Use CG utilities to compute the NICE iteration

the python utilities contain a `cg_utils` module that among other things has a Clebsch-Gordan class built to work using real spherical harmonics, which is introduced in the example `equivariant_demo.ipynb`. 

In [ ]:
CG = ClebschGordanReal(lmax=spherical_expansion_hypers["max_angular"])

the combination can also be done in bulk! This does it combining elementwise (basically computing the diagonal squares of the features)

In [ ]:
ref_feats.shape

In [ ]:
t1_bulk = CG.combine(ref_feats[...,lm_slice(3)], ref_feats[...,lm_slice(2)], 3)

In [ ]:
t1_bulk.shape

`ClebschGordanReal.combine_einsum()` provides a very flexible framework to perform all sorts of CG operations.
The two arguments are meant to have shape (...,-l1..l1) and (...,-l2..l2), i.e. the equivariant index is the last. 
Then, for each m1,m2,M term, the iteration calls einsum, according to the specified string
For instance, the NICE iteration
$$
\langle Q; nlk|\overline{\rho^{\otimes \nu+1}_i; \lambda\mu}\rangle = 
\sum_{m q} \langle n | \overline{\rho^{1}_i; lm}\rangle
\langle Q|\overline{\rho^{\otimes \nu}_i; kq}\rangle 
\langle lm; kq | \lambda\mu \rangle
$$
can be run as

In [ ]:
sel_l, sel_k, sel_lambda = 2,3,4
nice2_lambda = CG.combine_einsum(ref_feats[...,lm_slice(sel_l)], 
                                 ref_feats[...,lm_slice(sel_k)], 
                                 sel_lambda,
                                 combination_string="ian,iAN->ianAN")

... which results in a vector with shape (entry, element1, radial1, element2, radial2, -L..L)

In [ ]:
nice2_lambda[0].shape

cg_utils also provides a (naive) implementation that computes the whole NICE iteration at once. 
given two equivariants (i,...,LM) (i,...,LM) returns (i,...,...,l,k,LM) (with lots of zeros, because this is not optimized in any shape or form!)

In [ ]:
nice2_full = CG.combine_nice(ref_feats, ref_feats)

... which does the same as the `combine_einsum` above

In [ ]:
np.linalg.norm(nice2_full[..., sel_l, sel_k, lm_slice(sel_lambda)]-nice2_lambda)

# Compares NICE iteration with explicit definitions

The NICE iteration is a general case encompassing most of the widespread density correlation features

## SOAP

SOAP features are defined simply as
$$
\langle a_1 n_1; a_2 n_2; l | \overline{\rho_i^{\otimes 2}} \rangle = 
\frac{1}{\sqrt{2l+1}} \sum_m 
\langle a_1 n_1 l m | \rho_i \rangle  \langle a_2 n_2 l m | \rho_i \rangle^\star
$$

"complex-valued" definition from Bartók, PRB 2013

In [ ]:
soap_manual = np.zeros((ref_feats.shape[:3] + ref_feats.shape[1:3] + (spherical_expansion_hypers["max_angular"]+1,)) )
for l in range(spherical_expansion_hypers["max_angular"]+1):
    soap_manual[...,l] = np.real(np.einsum("ianm,iANm->ianAN",
              ref_feats[...,lm_slice(l)]@real2complex_matrix(l).T, 
              np.conjugate(ref_feats[...,lm_slice(l)]@real2complex_matrix(l).T)
             )) / np.sqrt(2*l+1)

In [ ]:
soap_manual[0,0,0,0,0]

the nice l=k, L=0 terms match these, except for an inconsequential $(-1)^l$ phase. 

In [ ]:
np.einsum("ii->i",nice2_full[0,0,0,0,0,:,:,0])

this is also true for the SOAP features computed directly from librascal - with a caveat that $a_1\ne a_2$ features are scaled by $\sqrt{2}$ as only half are computed and that indexing a specific block is a pain

In [ ]:
soap_hypers = deepcopy(spherical_expansion_hypers)

soap_hypers["soap_type"] = "PowerSpectrum"
soap_hypers["normalize"] = False

soap = SphericalInvariants(**soap_hypers)
soap_feats = soap.transform(selframe).get_features(soap)*feat_scaling**2  # scale

In [ ]:
soap_feats[0,:spherical_expansion_hypers["max_angular"]+1]

In [ ]:
soap_feats[0,spherical_expansion_hypers["max_radial"]**2*(spherical_expansion_hypers["max_angular"]+1):][:(spherical_expansion_hypers["max_angular"]+1)]

In [ ]:
soap_manual[0,0,0,1,0]*np.sqrt(2)

## $\lambda$-SOAP

The definition of $\lambda$-SOAP features is a pretty literal version of the $\nu=2$ NICE iteration,
$$
\langle a_1 n_1 l_1; a_2 n_2; l_2 | \overline{\rho_i^{\otimes 2}; \lambda\mu} \rangle = 
\sum_{m_1 m_2} 
\langle a_1 n_1 l_1 m_1 | \rho_i \rangle  \langle a_2 n_2 l_2 m_2 | \rho_i \rangle
\langle l_1m_1; l_2 m_2 | \lambda\mu \rangle
$$

In [ ]:
lsoap_manual = np.zeros((ref_feats.shape[:3] + (spherical_expansion_hypers["max_angular"]+1,) +
                         ref_feats.shape[1:3] + (spherical_expansion_hypers["max_angular"]+1,) +
                        (2*sel_lambda+1,)  ))
# NB: we convert to complex valued and pick the raw CG that is a (2l1+1,2l2+1,2L+1) matrix.
# also we pick real + imag because depending on parity the meaningful term might be in the real or imaginary part
for l1 in range(soap_hypers["max_angular"]+1):
    for l2 in range(soap_hypers["max_angular"]+1):
        if (l1, l2, sel_lambda) in CG._cgraw:
            res = np.einsum("ianm,iANM,mMW->ianANW",
              ref_feats[...,lm_slice(l1)]@real2complex_matrix(l1).T, 
              ref_feats[...,lm_slice(l2)]@real2complex_matrix(l2).T,
              CG._cgraw[(l1, l2, sel_lambda)]
             )@np.conjugate(real2complex_matrix(sel_lambda))
            lsoap_manual[:,:,:,l1,:,:,l2] = res.real + res.imag

as usual, the most tricky bit is indexing - lambda soap goes anl,anl, while in nice we store ananll

In [ ]:
lsoap_manual[0, 0,0,2, 0,0,3]

In [ ]:
nice2_full[0, 0,0, 0,0, 2,3, lm_slice(sel_lambda)]

indexing hell is just to align the layout - but the two arrays contain identical values

In [ ]:
np.linalg.norm(lsoap_manual - 
               np.moveaxis(nice2_full[...,lm_slice(sel_lambda)],(0,1,2,5,3,4,6,7),(0,1,2,3,4,5,6,7)))/np.linalg.norm(lsoap_manual)

there is also a more concise demo function that computes a (nearly) minimal set of lambda-soap features

In [ ]:
lsoap_utils = compute_lambda_soap(ref_feats, CG, sel_lambda, 0)

In [ ]:
lsoap_manual.shape

In [ ]:
lsoap_utils.shape

the two vectors have the same norm (and contain same info) but the one from `compute_lambda_soap` avoids lots of zeros

In [ ]:
np.linalg.norm(lsoap_manual)

In [ ]:
np.linalg.norm(lsoap_utils)

In [ ]:
len(lsoap_manual.flatten())

In [ ]:
len(lsoap_utils.flatten())

## Bispectrum

The bispectrum is basically the third-order NICE invariant. Usual definition is
$$
\langle a_1 n_1 l_1; a_2 n_2 l_2; a_3 n_3 l_3 | \overline{\rho_i^{\otimes 3};} \rangle = 
\frac{ (-1)^{l_3}}{\sqrt{2l_3+1}}
\sum_{m_1 m_2 m_3} 
\langle a_1 n_1 l_1 m_1 | \rho_i \rangle  \langle a_2 n_2 l_2 m_2 | \rho_i \rangle
\langle l_1m_1; l_2 m_2 | l_3 m_3 \rangle ^\star
$$

In [ ]:
bisp_manual = np.zeros(ref_feats.shape[:3] + (spherical_expansion_hypers["max_angular"]+1,) +
                         ref_feats.shape[1:3] + (spherical_expansion_hypers["max_angular"]+1,) +
                         ref_feats.shape[1:3] + (spherical_expansion_hypers["max_angular"]+1,) )
# NB: we convert to complex valued and pick the raw CG that is a (2l1+1,2l2+1,2L+1) matrix.
# also we pick real + imag because depending on parity the meaningful term might be in the real or imaginary part
for l1 in range(soap_hypers["max_angular"]+1):
    for l2 in range(soap_hypers["max_angular"]+1):
        for l3 in range(soap_hypers["max_angular"]+1):
            if (l1, l2, l3) in CG._cgraw:
                res = np.einsum("ianm,iANM,ibpW,mMW->ianANbp",
                  ref_feats[...,lm_slice(l1)]@real2complex_matrix(l1).T, 
                  ref_feats[...,lm_slice(l2)]@real2complex_matrix(l2).T,
                  np.conjugate(ref_feats[...,lm_slice(l3)]@real2complex_matrix(l3).T),              
                  CG._cgraw[(l1, l2, l3)]
                 )*(-1)**l3/np.sqrt(2*l3+1)
                bisp_manual[:, :,:,l1, :,:,l2, :,:,l3] = res.real + res.imag

Can be computed with a single loop based on $\nu=2$ NICE features. Given we only want the $\lambda=0$ equivariant, we don't call `combine_nice` but hardcode the only bit we need

In [ ]:
bisp_nice = np.zeros(ref_feats.shape[:3] + (spherical_expansion_hypers["max_angular"]+1,) +
                         ref_feats.shape[1:3] + (spherical_expansion_hypers["max_angular"]+1,) +
                         ref_feats.shape[1:3] + (spherical_expansion_hypers["max_angular"]+1,) )
for l in range(soap_hypers["max_angular"]+1):
    # while we are at it, we also reorder the indices in a bispectrum-like way
    bisp_nice[...,l] = CG.combine_einsum(nice2_full[...,lm_slice(l)], ref_feats[...,lm_slice(l)], 0, "ianANlL,ibp->ianlANLbp" )[...,0]

... and equal they are (apart from noise)

In [ ]:
bisp_nice[0,0,0,2,0,0,2,0]

In [ ]:
bisp_manual[0,0,0,2,0,0,2,0]

In [ ]:
np.linalg.norm(bisp_manual-bisp_nice)/np.linalg.norm(bisp_nice)

# Sum rule

One way to check the amount of information that is lost due to truncation of the NICE iteration (or of the basis set!) is to apply the sum rule from Goscinski et al. (2021), which is a consequence of the orthogonality of CG coefficients

Massive information loss comes due to the truncation of the NICE iteration to `max_angular`

In [ ]:
nice2_norm = np.linalg.norm(nice2_full[1])
nice1_norm = np.linalg.norm(ref_feats[1])
print("Residual variance: ", 1-nice2_norm/nice1_norm**2)

We can "simulate" a complete NICE iteration by taking only the first few terms of the expansion coefficients

In [ ]:
lcut=2
nice2_norm = np.linalg.norm(nice2_full[1,...,:lcut+1,:lcut+1,:(2*lcut+1)**2])
nice1_norm = np.linalg.norm(ref_feats[1,...,:(lcut+1)**2])
print("Residual variance: ", 1-nice2_norm/nice1_norm**2)

Truncation errors accumulate with successive iterations - so that the residual variance at $\nu=2$ is approximately twice that at $\nu=1$. This underscores the importance of getting the best possible basis set at low body order.

In [ ]:
lcut=2; ncut=4
nice2_norm_cut = np.linalg.norm(nice2_full[1,:,:ncut,:,:ncut,:lcut+1,:lcut+1,:(2*lcut+1)**2])
nice1_norm_cut = np.linalg.norm(ref_feats[1,:,:ncut,:(lcut+1)**2])
print("Residual variance (nu=1): ", 1-nice1_norm_cut/nice1_norm)
print("Residual variance (nu=2): ", 1-nice2_norm_cut/nice1_norm**2)